In [2]:
import os
import sys
import numpy as np
import argparse
import random

In [3]:
sys.path.append("../../")

In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
import torch

In [6]:
k_model = keras.models.load_model("../keras_models/pruned_model_copy_weights_from_retrained_model_20240124123209_acc_95.45455169677734_v3.keras")

2024-01-31 17:03:51.643493: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 17:03:51.644062: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 17:03:51.644122: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
state = torch.load("../retrained_models_after_pruned/retrained_model_20240124123209_acc_95.45455169677734_795th_epoch.pt")
t_weights = state['weight']

In [8]:
t_weights.keys()

odict_keys(['sfeb.0.weight', 'sfeb.1.weight', 'sfeb.1.bias', 'sfeb.1.running_mean', 'sfeb.1.running_var', 'sfeb.1.num_batches_tracked', 'sfeb.3.weight', 'sfeb.4.weight', 'sfeb.4.bias', 'sfeb.4.running_mean', 'sfeb.4.running_var', 'sfeb.4.num_batches_tracked', 'tfeb.0.weight', 'tfeb.1.weight', 'tfeb.1.bias', 'tfeb.1.running_mean', 'tfeb.1.running_var', 'tfeb.1.num_batches_tracked', 'tfeb.4.weight', 'tfeb.5.weight', 'tfeb.5.bias', 'tfeb.5.running_mean', 'tfeb.5.running_var', 'tfeb.5.num_batches_tracked', 'tfeb.7.weight', 'tfeb.8.weight', 'tfeb.8.bias', 'tfeb.8.running_mean', 'tfeb.8.running_var', 'tfeb.8.num_batches_tracked', 'tfeb.11.weight', 'tfeb.12.weight', 'tfeb.12.bias', 'tfeb.12.running_mean', 'tfeb.12.running_var', 'tfeb.12.num_batches_tracked', 'tfeb.14.weight', 'tfeb.15.weight', 'tfeb.15.bias', 'tfeb.15.running_mean', 'tfeb.15.running_var', 'tfeb.15.num_batches_tracked', 'tfeb.18.weight', 'tfeb.19.weight', 'tfeb.19.bias', 'tfeb.19.running_mean', 'tfeb.19.running_var', 'tfeb.19.

In [9]:
k_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 1, 15109, 5)       45        
                                                                 
 batch_normalization_12 (Ba  (None, 1, 15109, 5)       20        
 tchNormalization)                                               
                                                                 
 re_lu_12 (ReLU)             (None, 1, 15109, 5)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 7553, 32)       800       
                                                                 
 batch_normalization_13 (Ba  (None, 1, 7553, 32)       128       
 tchNormalization)                                         

In [21]:
t2k_pos_move = (2,3,1,0)

## sfeb.0.weight vs. conv2d_12

In [26]:
t_sfeb_0_weight = t_weights['sfeb.0.weight']
print(f"t_weights['sfeb.0.weight'] original shape:{t_sfeb_0_weight.shape}")
t_sfeb_0_weight = torch.permute(t_sfeb_0_weight,t2k_pos_move)
print(f"t_sfeb_0_weight shape after permute:{t_sfeb_0_weight.shape}")
k_conv2d_12_layer = k_model.get_layer('conv2d_12')
print(f"k_model.get_layer('conv2d_12'). shape:{type(k_conv2d_12_layer)}")
print(f"k_conv2d_12_layer weight:\n{k_model.get_layer('conv2d_12').get_weights()}")
print(f"t_sfeb_0_weight:\n{t_sfeb_0_weight}")

t_weights['sfeb.0.weight'] original shape:torch.Size([5, 1, 1, 9])
t_sfeb_0_weight shape after permute:torch.Size([1, 9, 1, 5])
k_model.get_layer('conv2d_12'). shape:<class 'keras.src.layers.convolutional.conv2d.Conv2D'>
k_conv2d_12_layer weight:
[array([[[[ 0.14281216,  0.17640175, -0.4899664 ,  0.11559714,
          -0.1485947 ]],

        [[ 0.21973214,  0.03517931, -0.5627965 ,  0.11975447,
          -0.2087061 ]],

        [[ 0.09018669,  0.00690036,  0.02574774, -0.24218576,
          -0.03620813]],

        [[ 0.7725846 , -0.24780475,  0.5764529 ,  0.37982678,
           0.22225192]],

        [[-0.21710275, -0.03813766,  0.08965266, -0.8616742 ,
           0.08274575]],

        [[-0.3066278 , -0.2278304 ,  0.2853256 ,  0.7015828 ,
           0.07163105]],

        [[-0.36687186,  0.16276164, -0.26189068,  0.10729038,
          -0.2701285 ]],

        [[-0.10925402,  0.02150343,  0.0166929 , -0.97063   ,
           0.09517566]],

        [[-0.04914073, -0.41611147,  0.0854072 ,

## ('sfeb.1.weight', 'sfeb.1.bias', 'sfeb.1.running_mean', 'sfeb.1.running_var') vs. batch_normalization_12

In [31]:
t_sfeb_1_weight = t_weights['sfeb.1.weight']
t_sfeb_1_bias = t_weights['sfeb.1.bias']
t_sfeb_1_runing_mean = t_weights['sfeb.1.running_mean']
t_sfeb_1_running_var = t_weights['sfeb.1.running_var']
k_conv2d_12_layer = k_model.get_layer('batch_normalization_12')
print(f"k_conv2d_12_layer weight:\n{k_conv2d_12_layer.get_weights()}")
print(f"t_sfeb_1_weight:\n{t_sfeb_1_weight}")
print(f"t_sfeb_1_weight:\n{t_sfeb_1_bias}")
print(f"t_sfeb_1_runing_mean :\n{t_sfeb_1_runing_mean}")
print(f"t_sfeb_1_running_var :\n{t_sfeb_1_running_var}")

k_conv2d_12_layer weight:
[array([0.7654353 , 0.5080408 , 0.6432544 , 0.5626859 , 0.51786804],
      dtype=float32), array([0.0662422 , 0.04040214, 0.52289575, 0.60694873, 0.08088733],
      dtype=float32), array([ 0.00029895, -0.00090161, -0.00040593, -0.00021506,  0.00066517],
      dtype=float32), array([0.03532692, 0.01011292, 0.0472468 , 0.00703194, 0.01624621],
      dtype=float32)]
t_sfeb_1_weight:
tensor([0.7654, 0.5080, 0.6433, 0.5627, 0.5179], device='cuda:0')
t_sfeb_1_weight:
tensor([0.0662, 0.0404, 0.5229, 0.6069, 0.0809], device='cuda:0')
t_sfeb_1_runing_mean :
tensor([ 0.0003, -0.0009, -0.0004, -0.0002,  0.0007], device='cuda:0')
t_sfeb_1_running_var :
tensor([0.0353, 0.0101, 0.0472, 0.0070, 0.0162], device='cuda:0')


## sfeb.3.weight vs. conv2d_13

In [45]:
t_sfeb_3_weight = t_weights['sfeb.3.weight']
print(f"t_weights['sfeb.3.weight'] original shape:{t_sfeb_3_weight.shape}")
t_sfeb_3_weight = torch.permute(t_sfeb_3_weight,t2k_pos_move)
print(f"t_sfeb_3_weight shape after permute:{t_sfeb_3_weight.shape}")
k_conv2d_13_layer = k_model.get_layer('conv2d_13')
print(f"k_conv2d_12_layer weight:\n{np.asarray(k_conv2d_13_layer.get_weights()).shape}")
print(f"t_sfeb_0_weight:\n{len(t_sfeb_3_weight[0][0][0][:])}")

t_weights['sfeb.3.weight'] original shape:torch.Size([32, 5, 1, 5])
t_sfeb_3_weight shape after permute:torch.Size([1, 5, 5, 32])
k_conv2d_12_layer weight:
(1, 1, 5, 5, 32)
t_sfeb_0_weight:
32
